In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
batch_size = 64  
epochs = 100  
latent_dim = 256 .
num_samples = 10000 
n_features = 50
data_path = "mar.txt"

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [5]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 85
Max sequence length for inputs: 19
Max sequence length for outputs: 42


In [6]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [7]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [9]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [10]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 45s 324ms/step - loss: 1.5547 - accuracy: 0.6424 - val_loss: 1.5138 - val_accuracy: 0.6104
Epoch 2/100
125/125 [==============================] - 38s 300ms/step - loss: 1.1409 - accuracy: 0.7089 - val_loss: 1.1936 - val_accuracy: 0.6900
Epoch 3/100
125/125 [==============================] - 36s 290ms/step - loss: 0.9111 - accuracy: 0.7585 - val_loss: 1.0511 - val_accuracy: 0.7173
Epoch 4/100
125/125 [==============================] - 37s 297ms/step - loss: 0.8076 - accuracy: 0.7804 - val_loss: 0.9715 - val_accuracy: 0.7341
Epoch 5/100
125/125 [==============================] - 37s 297ms/step - loss: 0.7430 - accuracy: 0.7942 - val_loss: 0.8992 - val_accuracy: 0.7519
Epoch 6/100
125/125 [==============================] - 38s 307ms/step - loss: 0.6880 - accuracy: 0.8089 - val_loss: 0.8555 - val_accuracy: 0.7658
Epoch 7/100
125/125 [==============================] - 37s 294ms/step - loss: 0.6468 - accuracy: 0.8200 - val_loss: 0.8344 -

Epoch 57/100
125/125 [==============================] - 34s 275ms/step - loss: 0.0912 - accuracy: 0.9711 - val_loss: 0.9882 - val_accuracy: 0.8107
Epoch 58/100
125/125 [==============================] - 35s 276ms/step - loss: 0.0877 - accuracy: 0.9722 - val_loss: 0.9879 - val_accuracy: 0.8119
Epoch 59/100
125/125 [==============================] - 35s 279ms/step - loss: 0.0856 - accuracy: 0.9726 - val_loss: 0.9972 - val_accuracy: 0.8113
Epoch 60/100
125/125 [==============================] - 34s 271ms/step - loss: 0.0825 - accuracy: 0.9735 - val_loss: 1.0042 - val_accuracy: 0.8112
Epoch 61/100
125/125 [==============================] - 35s 278ms/step - loss: 0.0807 - accuracy: 0.9742 - val_loss: 1.0137 - val_accuracy: 0.8104
Epoch 62/100
125/125 [==============================] - 34s 270ms/step - loss: 0.0780 - accuracy: 0.9748 - val_loss: 1.0222 - val_accuracy: 0.8111
Epoch 63/100
125/125 [==============================] - 37s 300ms/step - loss: 0.0757 - accuracy: 0.9753 - val_loss: 1

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


In [11]:
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [12]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [13]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [14]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: जा.

-
Input sentence: Run!
Decoded sentence: धाव!

-
Input sentence: Run!
Decoded sentence: धाव!

-
Input sentence: Run!
Decoded sentence: धाव!

-
Input sentence: Run!
Decoded sentence: धाव!

-
Input sentence: Who?
Decoded sentence: कोण?

-
Input sentence: Wow!
Decoded sentence: वाह!

-
Input sentence: Duck!
Decoded sentence: खाली वाका!

-
Input sentence: Fire!
Decoded sentence: फायर!

-
Input sentence: Fire!
Decoded sentence: फायर!

-
Input sentence: Help!
Decoded sentence: वाचवा!

-
Input sentence: Help!
Decoded sentence: वाचवा!

-
Input sentence: Jump!
Decoded sentence: उडी मार!

-
Input sentence: Jump!
Decoded sentence: उडी मार!

-
Input sentence: Jump.
Decoded sentence: उडी मार.

-
Input sentence: Jump.
Decoded sentence: उडी मार.

-
Input sentence: Stop!
Decoded sentence: थांबा!

-
Input sentence: Stop!
Decoded sentence: थांबा!

-
Input sentence: Wait!
Decoded sentence: थांबा!

-
Input sentence: Wait!
Decoded sentence: थांबा!



For Machine Translation, a sequence to sequence model is used. The model consists of an encoder and a decoder each performing its specific purpose.

The encoder(RNN layer) processes the input sequence and returns its own internal state. We discard the outputs of the encoder RNN, only recovering the state. This state will serve as the "context", or "conditioning", of the decoder in the next step.

The decoder(also RNN layer), is trained to predict the next characters of the target sequence given previous characters of the target sequence. Encoder uses as initial state the state vectors from the encoder, which is how the decoder obtains information about what it is supposed to generate. Effectively, the decoder learns to generate targets.

We started by importing the txt file and creating 2 lists for the input texts and target texts respectively. We also created the 2 sets for the language characters.

We then assigned indices to the sorted list of both input characters and target characters. Then we prepare a 3D array, for all encoder input data, decoder input data and the target data. The decoder_target_data is the same as decoder_input_data but offset by one timestep. One-hot vectorization will then be done.

We then create our LSTM layer and pass our encoder tokens. However we do not use the encoder outputs. The same thing is done for the decoded tokens too and this is output is required by us. 

We then compile and train our model for 100 epochs which gives a good accuracy.

After training,
Encode the input sentence and retrieve the initial decoder state
Run one step of the decoder with this initial state and a "start of sequence" token as target. The output will be the next target character.
Append the target character predicted and repeat.